In [ ]:
import pandas as pd 
import plotly.graph_objects as go
import plotly.express as px
import sys, os
fn_data = "../data/renewables_with_load.parquet"
from data import get_generation, normalize_generation, get_profiles
from graphs import plot_profile, plot_daily_generation

In [18]:
df_gen = get_generation(fn_data, country="DE", year=2023)
df_norm = normalize_generation(df_gen, shares={"Wind": 0.4, "Solar": 0.4, "Baseload": 0.2}, total_demand=0)
profiles = get_profiles(df_norm)

In [42]:
def get_storage_stats(df:pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """Get curtailment and energy overshoot given the frame of generation
    
    Args:
        df: frame with hourly generation and demand data

    Returns:
        hourly dataframe with storage statistics; aggregated statistics
    """
    col_gen = [c for c in df.columns if c != "Demand"]
    df_ = df.assign(
        TotalSupply=lambda df: df[col_gen].sum(1),
        ExcessSupply=lambda df: df["TotalSupply"] - df["Demand"],
        Curtailment=lambda df: (df["ExcessSupply"] > 0).astype("int")*df["ExcessSupply"],
        LostLoad=lambda df: (-1)*(df["ExcessSupply"] < 0).astype("int")*df["ExcessSupply"],
    )
    total = df_.sum()
    stats = {
        var: {
            "Total": total[var],
            "TotalPercentOfDemand": total[var]/total["Demand"]*100,
            "Max": df_[var].max(),
            #"MaxPercent": df_.loc[df_[var].idxmax(), var]/df_.loc[df_[var].idxmax(), "Demand"]*100,
            "Hours": len(df_[df_[var] > 9])
        }
        for var in ["Curtailment", "LostLoad"]
    }
    return df_, pd.DataFrame.from_dict(stats).round(1)
df, stats = get_storage_stats(df_norm)
stats


,Curtailment,LostLoad
Total,102985351.2,102985351.2
TotalPercentOfDemand,22.5,22.5
Max,104309.3,55468.4
Hours,3542.0,5215.0
